In [1]:
import pandas as pd
import datetime

In [29]:
global base_date
base_date=None

def setidx(df):
    global base_date
    ilist=[]
    for i in range(len(df.index)):
        res=df.index[i]
        time_stamp = datetime.datetime.strptime(res.split(' ')[-1], '%H:%M')
        minute_index = round(time_stamp.minute/10)
        hour_index = time_stamp.hour
        if base_date == None:
            base_date = datetime.datetime.strptime(res.split(' ')[0], '%m/%d/%y')
        day = (datetime.datetime.strptime(res.split(' ')[0], '%m/%d/%y')-base_date).days
        time_index = day * 24 * 6 - 31 + minute_index + hour_index * 6
        ilist.append(time_index)
    df.index=ilist
    return df


origdf=pd.read_csv("Data/bridgeport1week1-train.csv") 
origdf=origdf.drop([0])
base_date = None
newdf=pd.DataFrame(origdf[origdf.columns[0:2]])
newdf.columns=["",newdf.columns[0]]
newdf=newdf.set_index(newdf.columns[0])
newdf=newdf.dropna()
newdf=setidx(newdf)
def gettindex(res, base_date):
    time_stamp = datetime.datetime.strptime(res.split(' ')[-1], '%H:%M')
    minute_index = round(time_stamp.minute/10)
    hour_index = time_stamp.hour
    if base_date == None:
        base_date = datetime.datetime.strptime(res.split(' ')[0], '%m/%d/%y')
    day = (datetime.datetime.strptime(res.split(' ')[0], '%m/%d/%y')-base_date).days
    time_index = day * 24 * 6 - 31 + minute_index + hour_index * 6
    return(time_index,base_date)

for i in range(1,13):
    tempdf=pd.DataFrame(origdf[origdf.columns[i*3:i*3+2]])
    tempdf.columns=["",tempdf.columns[0]]
    tempdf=tempdf.dropna()
    tempdf=tempdf.set_index(tempdf.columns[0])
    ilist=[]
    tempdf=setidx(tempdf)
    newdf=pd.concat([newdf,tempdf],axis=1)
    
origdf2=pd.read_csv("Data/bridgeport1week2-train.csv") 
origdf2=origdf2.drop([0])

newdf2=pd.DataFrame(origdf2[origdf2.columns[0:2]])
newdf2.columns=["",newdf2.columns[0]]
newdf2=newdf2.set_index(newdf2.columns[0])
newdf2=newdf2.dropna()
newdf2=setidx(newdf2)
for i in range(1,13):
    tempdf2=pd.DataFrame(origdf2[origdf2.columns[i*3:i*3+2]])
    tempdf2.columns=["",tempdf2.columns[0]]
    tempdf2=tempdf2.dropna()
    tempdf2=tempdf2.set_index(tempdf2.columns[0])
    tempdf2=setidx(tempdf2)
    newdf2=pd.concat([newdf2,tempdf2],axis=1)
resdf=pd.concat([newdf, newdf2])
resdf=resdf.replace(float("Nan"), -1)
resdf.to_csv('resdf.csv') 

In [28]:
# print(resdf.index)
print(newdf[newdf.columns[6]])

2/15/20 5:09          NaN
2/15/20 5:18    0.0525772
2/15/20 5:28    0.0503932
2/15/20 5:38    0.0428264
2/15/20 5:49    0.0557567
                  ...    
2/22/20 3:20          NaN
2/22/20 4:10          NaN
2/22/20 4:29          NaN
2/22/20 4:40          NaN
2/22/20 4:50          NaN
Name: Machines > Bridgeport Mill 1 > Spindle > X-Axial > Peak Acceleration, Length: 1561, dtype: object


In [15]:
print(newdf.head())
print(newdf2.head())
print(resdf.head())

             Machines > Bridgeport Mill 1 > Spindle > Y-Radial > Band 1  \
2/15/20 5:09                                          0.0024374           
2/15/20 5:18                                          0.0034169           
2/15/20 5:28                                          0.0034582           
2/15/20 5:38                                          0.0014652           
2/15/20 5:49                                          0.0031839           

             Machines > Bridgeport Mill 1 > Spindle > Y-Radial > Damage Accumulation  \
2/15/20 5:09                                          1.0124303                        
2/15/20 5:18                                           0.983099                        
2/15/20 5:28                                          0.9936279                        
2/15/20 5:38                                          0.9791234                        
2/15/20 5:49                                          0.9771959                        

             Machine

In [20]:
print(newdf.columns)

Index(['Machines > Bridgeport Mill 1 > Spindle > Y-Radial > Band 1'], dtype='object')


In [5]:
import numpy as np
import csv
import datetime
import json
import os
from glob import glob

if not os.path.exists('Preprocessed_Data'):
    os.mkdir('Preprocessed_Data')

csvs = glob('Data/*.csv')
for file_path in csvs:
    if not 'submit' in file_path:
        labels = []
        X = {}
        Y = {}
        base_date = None
        with open(file_path, 'r') as file:
            reader = csv.reader(file)
            for i,row in enumerate(reader):
                if i == 0:
                    for label in row:
                        if label != '':
                            labels.append(label)
                            if 'Damage' in label:
                                Y[label] = np.zeros([1008])-1
                            else:
                                X[label] = np.zeros([1008])-1

                elif i != 1:
                    for j,label in enumerate(labels):
                        if row[j*3] != '':

                            time_stamp = datetime.datetime.strptime(row[j*3].split(' ')[-1], '%H:%M')
                            minute_index = round(time_stamp.minute/10)
                            hour_index = time_stamp.hour
                            if base_date == None:
                                base_date = datetime.datetime.strptime(row[j*3].split(' ')[0], '%m/%d/%y')

                            day = (datetime.datetime.strptime(row[j*3].split(' ')[0], '%m/%d/%y')-base_date).days

                            time_index = day * 24 * 6 - 31 + minute_index + hour_index * 6

                            if 'Damage' in label:
                                Y[label][time_index] = float(row[j*3 + 1])
                            else:
                                X[label][time_index] = float(row[j*3 + 1])
            X_headers = []
            Y_headers = []
            for i,d in enumerate(X):
                X_headers.append(d)
                if i == 0:
                    X_np = np.expand_dims(X[d],-1)
                else:
                    X_np = np.concatenate([X_np,np.expand_dims(X[d],-1)],-1)

            for i,d in enumerate(Y):
                Y_headers.append(d)
                if i == 0:
                    Y_np = np.expand_dims(Y[d],-1)
                else:
                    Y_np = np.concatenate([Y_np,np.expand_dims(Y[d],-1)],-1)
            file_path = file_path.replace('Data','Preprocessed_Data')

            np.save(file_path.replace('.csv','-X.npy'),X_np)
            
            if not 'test' in file_path:
                np.save(file_path.replace('.csv','-Y.npy'),Y_np)

            with open(file_path.replace('.csv','-X-Headers.json'), 'w') as outfile:
                json.dump(X_headers, outfile)
            if not 'test' in file_path:
                with open(file_path.replace('.csv','-Y-Headers.json'), 'w') as outfile:
                    json.dump(Y_headers, outfile)


10
08


TypeError: function missing required argument 'day' (pos 3)

In [74]:
def gettindex(res)
    time_stamp = datetime.datetime.strptime(res.split(' ')[-1], '%H:%M')
    minute_index = round(time_stamp.minute/10)
    hour_index = time_stamp.hour
    if base_date == None:
        base_date = datetime.datetime.strptime(row[j*3].split(' ')[0], '%m/%d/%y')

    day = (datetime.datetime.strptime(row[j*3].split(' ')[0], '%m/%d/%y')-base_date).days

    time_index = day * 24 * 6 - 31 + minute_index + hour_index * 6
    return(time_index)

SyntaxError: invalid syntax (<ipython-input-74-7645ee1f46e4>, line 1)

5:09:
